In [1]:
import pandas as pd
import requests 
import time
import pyodbc
# to handle  data retrieval
import urllib3
from urllib3 import request
# to handle certificate verification
import certifi
# uncomment below if installation needed (not necessary in Colab)
#!pip install certifi
import json
import numpy as np
import requests
#from urllib2 import Request, urlopen

# import urllib library
from urllib.request import urlopen

In [2]:
#this verifies that the website is secure and has a valid certificate
http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

Grab NDC Codes from API

In [3]:
# get data from the API
url = 'https://rxnav.nlm.nih.gov/REST/allNDCstatus.json?status=active'
r = http.request('GET', url)
r.status

200

In [4]:
#this translates the json code to a dictionary
response = requests.get(url)
maindict = response.json()

In [5]:
#verify the datatype
print(type(maindict))

<class 'dict'>


Iterate NDCs to List

In [6]:
#this extracts the dictionary out of the dictionary then converts to a list
for key in maindict.keys():
        seconddict = maindict.get(key)
        datalist = seconddict['ndc']

In [7]:
#verify the datatype
print(type(datalist))

<class 'list'>


In [8]:
print(len(datalist))

255970


In [9]:
#convert list to dataframe
dfdata = pd.DataFrame(datalist, columns=['ndc'])

Run NDC list through NDCStatus API 

In [48]:
from urllib.parse import urlparse
import ast
import itertools
starttime=time.time()
datalen = len(datalist)
#append = []
ndcstatus = []
ndcstatushistory = []
ndcstatussource = []

df = pd.DataFrame()

def gloriouspurpose (start, numrows):
    for i in range(start, numrows):
        ndcnum=datalist[i]
        print('index ' + str(ndcnum) + ' timeperindex '+str((time.time()-starttime)))
        # get data from the API
        url2 = ("https://rxnav.nlm.nih.gov/REST/ndcstatus.json?ndc=" + str(ndcnum))
        r = http.request('GET', url2)
        #if this gives a code 200, then everything processed right
        r.status
        #this translates the json code to a dictionary
        response = requests.get(url2)
        maindict2 = response.json()
         #this extracts the dictionary out of the dictionary then converts to a list
        for key in maindict2.keys():
            seconddict2 = maindict2.get(key)
            historylist = seconddict2.get('ndcHistory')
             #grabbing items in the historylist
            activeRxcui = [i["activeRxcui"] for i in historylist]
            originalRxcui = [i["originalRxcui"] for i in historylist]
            startDateRxcui = [i["startDate"] for i in historylist]
            endDateRxcui = [i["endDate"] for i in historylist]
            
             #This builds the ndchistorydf 
            ndcHistorydict = {'ndc11':ndcnum,'activeRxcui':activeRxcui, 'originalRxcui':originalRxcui, 'startDateRxcui':startDateRxcui, 'endDateRxcui':endDateRxcui}
            ndcHistorylist = list(ndcHistorydict.values())
            ndcstatushistory.append(ndcHistorylist)
             #historydf=pd.DataFrame(ndcstatushistory,columns=['ndc11','activeRxcui', 'originalRxcui', 'startDateRxcui', 'endDateRxcui'])
            
             #This builds the sourcenamedf
            sourcedict = seconddict2.get('sourceList')
            sourcelist = list(map(list, (ele for ele in sourcedict.values())))
            sourcelist2 = str(sourcelist)[1:-1]
            sourcestr = str(sourcelist2)[1:-1]
            sourcenamedict = {'ndc11':ndcnum,'sourcename':sourcestr}
            sourcelist3 = list(sourcenamedict.values())
            ndcstatussource.append(sourcelist3)
            #sourcedf=pd.DataFrame(ndcstatussource,columns=['ndc11','sourceName'])
            
            #buld the othervardf
            dictlist = list(seconddict2.items())
            outsidevar = ('ndc11','conceptStatus', 'conceptName', 'comment', 'altNdc', 'status',  'rxnormNdc', 'active')
            dictlist = {key: seconddict2[key] for key in seconddict2.keys()
                            & {'ndc11','conceptStatus', 'conceptName', 'comment', 'altNdc', 'status',  'rxnormNdc', 'active'}}
            othervarlist = list(dictlist.values())
            ndcstatus.append(othervarlist)
            #othervardf=pd.DataFrame(ndcstatus,columns=['comment','rxnormNdc', 'altNdc', 'ndc11', 'active', 'conceptName',  'conceptStatus', 'status'])

In [ ]:
gloriouspurpose(0, 20000)

index 00002140701 timeperindex 0.8090205192565918
index 00002143301 timeperindex 2.297053098678589
index 00002143361 timeperindex 3.3340742588043213
index 00002143380 timeperindex 4.342101097106934
index 00002143401 timeperindex 5.373137712478638
index 00002143461 timeperindex 6.3901591300964355
index 00002143480 timeperindex 7.4091737270355225
index 00002143601 timeperindex 8.424195051193237
index 00002143611 timeperindex 9.470265626907349
index 00002143661 timeperindex 10.45932936668396
index 00002144501 timeperindex 11.478395462036133
index 00002144509 timeperindex 12.571488618850708
index 00002144511 timeperindex 13.60456919670105
index 00002144527 timeperindex 14.642583131790161
index 00002144561 timeperindex 27.73730206489563
index 00002197561 timeperindex 28.762404680252075
index 00002197590 timeperindex 30.681596517562866
index 00002223601 timeperindex 34.76800513267517
index 00002223661 timeperindex 35.79310750961304
index 00002223680 timeperindex 36.81420946121216
index 00002

index 00002775201 timeperindex 196.20514726638794
index 00002775205 timeperindex 197.205246925354
index 00002791001 timeperindex 198.18934559822083
index 00002795001 timeperindex 199.18844556808472
index 00002803101 timeperindex 200.16954350471497
index 00002814701 timeperindex 201.1566424369812
index 00002814801 timeperindex 202.1687433719635
index 00002814901 timeperindex 203.1818447113037
index 00002820701 timeperindex 204.1859450340271
index 00002820705 timeperindex 205.16804337501526
index 00002820761 timeperindex 206.47117376327515
index 00002821301 timeperindex 207.49227571487427
index 00002821305 timeperindex 208.50237679481506
index 00002821401 timeperindex 208.96942329406738
index 00002821405 timeperindex 209.4154679775238
index 00002821501 timeperindex 210.4195683002472
index 00002821517 timeperindex 211.51867818832397
index 00002821599 timeperindex 212.5567820072174
index 00002822201 timeperindex 213.59288573265076
index 00002822259 timeperindex 214.62198853492737
index 000

index 00004641501 timeperindex 375.68709349632263
index 00004694004 timeperindex 376.73619842529297
index 00005010001 timeperindex 377.7523000240326
index 00005010002 timeperindex 378.7674014568329
index 00005010005 timeperindex 380.0875337123871
index 00005010010 timeperindex 381.10163474082947
index 00005197101 timeperindex 382.1267373561859
index 00005197102 timeperindex 382.75780057907104
index 00005197104 timeperindex 383.36886167526245
index 00005197105 timeperindex 383.9919240474701
index 00005250002 timeperindex 384.6299877166748
index 00005250023 timeperindex 385.67109179496765
index 00005250033 timeperindex 386.7261972427368
index 00005550919 timeperindex 387.2372486591339
index 00005550924 timeperindex 388.27435207366943
index 00005550925 timeperindex 389.41746640205383
index 00005551019 timeperindex 390.44656920433044
index 00005551020 timeperindex 391.49467420578003
index 00005551119 timeperindex 392.5197765827179
index 00005551750 timeperindex 393.55988073349
index 000055

index 00006307603 timeperindex 549.2804510593414
index 00006307604 timeperindex 550.3135545253754
index 00006308001 timeperindex 551.3356566429138
index 00006351458 timeperindex 552.3417572975159
index 00006351659 timeperindex 553.4128410816193
index 00006351936 timeperindex 554.5099396705627
index 00006360301 timeperindex 555.2980051040649
index 00006360360 timeperindex 556.2150473594666
index 00006360361 timeperindex 557.248083114624
index 00006380001 timeperindex 558.31010389328
index 00006380018 timeperindex 559.305139541626
index 00006380101 timeperindex 560.3311772346497
index 00006380118 timeperindex 561.4181966781616
index 00006382210 timeperindex 562.4452140331268
index 00006382310 timeperindex 563.5522372722626
index 00006384114 timeperindex 564.665256023407
index 00006384130 timeperindex 565.6992766857147
index 00006384371 timeperindex 566.7502963542938
index 00006384501 timeperindex 567.8173155784607
index 00006384571 timeperindex 568.9073355197906
index 00006385601 timeper

index 00007338713 timeperindex 723.0235981941223
index 00007338813 timeperindex 724.0426216125488
index 00007365022 timeperindex 725.0646681785583
index 00007365030 timeperindex 726.1067724227905
index 00007413920 timeperindex 727.1638383865356
index 00007414020 timeperindex 728.2668840885162
index 00007414120 timeperindex 729.6419584751129
index 00007414220 timeperindex 730.6770610809326
index 00007440701 timeperindex 731.7201564311981
index 00007488213 timeperindex 732.748245716095
index 00007488313 timeperindex 733.8273410797119
index 00007488513 timeperindex 734.8554286956787
index 00007488713 timeperindex 735.91752409935
index 00007488813 timeperindex 736.9606237411499
index 00007488859 timeperindex 738.272735118866
index 00007489020 timeperindex 739.3707973957062
index 00007489120 timeperindex 739.8778300285339
index 00007489220 timeperindex 740.9728755950928
index 00007489320 timeperindex 742.0119082927704
index 00007489420 timeperindex 743.0849266052246
index 00007489520 timepe

...

index 00113022489 timeperindex 6021.484434127808
index 00113022491 timeperindex 6022.590544939041
index 00113022771 timeperindex 6023.625648498535
index 00113023462 timeperindex 6025.439829826355
index 00113024101 timeperindex 6026.532938957214
index 00113025968 timeperindex 6027.349020719528
index 00113027262 timeperindex 6028.384124279022
index 00113027408 timeperindex 6029.451230764389
index 00113027468 timeperindex 6030.488334655762
index 00113027953 timeperindex 6031.527438402176
index 00113027962 timeperindex 6032.522537946701
index 00113028273 timeperindex 6033.529638767242
index 00113028502 timeperindex 6034.578743696213
index 00113028826 timeperindex 6035.605846405029
index 00113029126 timeperindex 6036.669952630997
index 00113029626 timeperindex 6037.703055858612
index 00113029827 timeperindex 6038.763161897659
index 00113029858 timeperindex 6039.795264959335
index 00113029860 timeperindex 6040.830368518829
index 00113029962 timeperindex 6041.881473779678
index 00113030234 ti

index 00113060826 timeperindex 6219.344218254089
index 00113060828 timeperindex 6220.421325922012
index 00113061239 timeperindex 6221.485432386398
index 00113061246 timeperindex 6222.580541610718
index 00113061249 timeperindex 6223.650648832321
index 00113061258 timeperindex 6224.732757091522
index 00113061260 timeperindex 6225.793863296509
index 00113061265 timeperindex 6226.851968765259
index 00113061275 timeperindex 6227.899073600769
index 00113061288 timeperindex 6228.970180749893
index 00113062264 timeperindex 6230.0352873802185
index 00113062326 timeperindex 6231.066390275955
index 00113062862 timeperindex 6232.416525363922
index 00113062871 timeperindex 6234.218705415726
index 00113062878 timeperindex 6236.014885187149
index 00113064526 timeperindex 6237.347018241882
index 00113064762 timeperindex 6238.437127351761
index 00113064771 timeperindex 6240.2313067913055
index 00113064776 timeperindex 6242.053488969803
index 00113064778 timeperindex 6243.854669094086
index 00113064782 

index 00113243262 timeperindex 6403.76765871048
index 00113246162 timeperindex 6404.797761917114
index 00113247662 timeperindex 6405.803862333298
index 00113365074 timeperindex 6406.803962469101
index 00113436862 timeperindex 6407.82706451416
index 00113436871 timeperindex 6408.8761694431305
index 00113436876 timeperindex 6409.9562776088715
index 00113436878 timeperindex 6411.040385723114
index 00113436879 timeperindex 6412.117493629456
index 00113450053 timeperindex 6413.186600446701
index 00113450062 timeperindex 6414.231705188751
index 00113600201 timeperindex 6415.276809453964
index 00113600203 timeperindex 6416.310912847519
index 00113700078 timeperindex 6417.339015722275
index 00113700079 timeperindex 6419.146196603775
index 00113700275 timeperindex 6420.952377080917
index 00113700287 timeperindex 6422.048486709595
index 00113700301 timeperindex 6423.131595134735
index 00113700303 timeperindex 6424.186700344086
index 00113700478 timeperindex 6425.248806715012
index 00113700501 ti

index 00113826458 timeperindex 6582.045484781265
index 00113826460 timeperindex 6582.82556271553
index 00113895926 timeperindex 6583.872667312622
index 00113900100 timeperindex 6584.726752758026
index 00113945813 timeperindex 6585.525832653046
index 00113945839 timeperindex 6586.360916137695
index 00113945866 timeperindex 6587.219002246857
index 00113945895 timeperindex 6588.062086343765
index 00113949062 timeperindex 6588.918171882629
index 00113949071 timeperindex 6589.770257234573
index 00113949078 timeperindex 6590.685348749161
index 00113975565 timeperindex 6591.584438562393
index 00115051101 timeperindex 6592.368516921997
index 00115052201 timeperindex 6593.133593320847
index 00115052202 timeperindex 6593.894669532776
index 00115053301 timeperindex 6594.645744562149
index 00115053302 timeperindex 6595.405820608139
index 00115067151 timeperindex 6596.177897691727
index 00115067250 timeperindex 6596.95597577095
index 00115069151 timeperindex 6597.740054130554
index 00115069250 time

index 00115148801 timeperindex 6742.762554883957
index 00115148901 timeperindex 6743.797658443451
index 00115149001 timeperindex 6744.815760135651
index 00115149101 timeperindex 6745.848863601685
index 00115151048 timeperindex 6746.876966238022
index 00115151058 timeperindex 6747.612039804459
index 00115151063 timeperindex 6748.648143291473
index 00115151245 timeperindex 6749.715250015259
index 00115151246 timeperindex 6750.741352796555
index 00115151258 timeperindex 6751.767455339432
index 00115151764 timeperindex 6752.778556346893
index 00115151765 timeperindex 6753.803658723831
index 00115151866 timeperindex 6754.843762636185
index 00115152210 timeperindex 6756.151893615723
index 00115152310 timeperindex 6757.185997009277
index 00115152501 timeperindex 6758.1780960559845
index 00115152608 timeperindex 6759.215199947357
index 00115152668 timeperindex 6760.251303434372
index 00115152708 timeperindex 6761.274405956268
index 00115152768 timeperindex 6762.3005084991455
index 00115152808 

In [ ]:
historydf=pd.DataFrame(ndcstatushistory,columns=['ndc11','activeRxcui', 'originalRxcui', 'startDateRxcui', 'endDateRxcui'])
sourcedf=pd.DataFrame(ndcstatussource,columns=['ndc11','sourceName'])
othervardf=pd.DataFrame(ndcstatus,columns=['comment','rxnormNdc', 'altNdc', 'ndc11', 'active', 'conceptName',  'conceptStatus', 'status'])

In [ ]:
historydf.head(20)

In [ ]:
sourcedf.head(20)

In [ ]:
othervardf.head(20)

In [ ]:
#### The merges all the dataframes together using NDC11 as primary key ###
df1 = pd.merge(
    historydf,
    sourcedf,
    how="outer",
    on='ndc11')
ndc_statusdf = pd.merge(
    othervardf,
    df1,
    how="outer",
    on='ndc11')
ndc_statusdf.head(10)

In [ ]:
### This will test the output ####
ndc_statusdf.to_csv('ndc_statusdf.csv', index=False)

### Below is the initial database creation #####
#import pyodbc  
#connection = pyodbc.connect('Driver={SQL Server};'
                      #'Server=XXXXXXXX;'
                      #'Database=XXXXXX;'
                      #'Trusted_Connection=yes;')

#cursor = connection.cursor()
         
        
# Create a new record
    #sql = "INSERT INTO `XXXXXX` (`comment`, `rxnormNdc`, `altNdc`, `ndc11`, `active`, `conceptName`, 'status', 'activeRxcui', 'originalRxcui', 'startDateRxcui', 'endDateRxcui', 'sourceName') VALUES (%s, %s, %s, %s, %s, %s)"
    #cursor.execute(sql, ndc_statusdf)

    # connection is not autocommit by default. So we must commit to save our changes.
    #connection.commit()

#except Error as e:
    #print(e)

#finally:
    # close the database connection using close() method.
    #connection.close()